## Intro: This is a Markdown cell

Markdown is a simple language to add basic formatting to plain text. Cells in a Jupyter notebook can be changed into Markdown cells using the *Cell > Cell Type > Markdown* menu. You can double click within the cell to edit/view source, and **run** the cell to produce the formatted text. Take a look at the source for this cell, which includes bold/italic text, headers, links, and code blocks.

Next, we'll tackle a common problem in setting up molecular dynamics simulations from crystal or cryo structures:
modeling missing loops where there is no clear electron density (perhaps due to flexibility or multiple conformations).

We'll use next-generation KIC: https://doi.org/10.1371/journal.pone.0063090  
https://www.rosettacommons.org/docs/latest/application_documentation/structure_prediction/loop_modeling/next-generation-KIC  
within the [Rosetta loopmodel application](https://www.rosettacommons.org/docs/latest/application_documentation/structure_prediction/loop_modeling/loopmodel).

For loopmodel, we need to add initial coordinates for the missing atoms to the pdb file, and they need to be linearly independent. Note: this is not required for the remodel application, but we would not be able to use next-generation KIC or KIC with fragments, which perform better on benchmarks.

We will also need to generate a loop file. From the Rosetta documentation, it should contain:

    column1  "LOOP":     The loop file identify tag
    column2  "integer":  Loop start residue number
    column3  "integer":  Loop end residue number
    column4  "integer":  Cut point residue number, >=startRes, <=endRes. Default: 0 (let the loop modeling code 
                         choose the cut point)
                         Note: Setting the cut point outside the loop can lead to a segmentation fault. 
    column5  "float":    Skip rate. Default: 0 (never skip modeling this loop)
    column6  "boolean":  Extend loop (i.e. discard the native loop conformation and rebuild the loop from
                         scratch, idealizing all bond lengths and angles). Default: 0 (false)

And while we're at it, we might as well automatically generate the batch scripts we'll need to run loopmodel and submit jobs on our karplus cluster using [GNU parallel](https://www.gnu.org/software/parallel/) (I also have GNU parallel and Rosetta compiled on Savio).

In [1]:
# Let's start with a more manual method. We'll want to generate random numbers for the fake atom positions, so
# we'll import a uniform random number generator:
from random import uniform
# Note: we don't care about the numbers being actually random, we just need to change them all from (0,0,0), so we
# won't bother seeding the number generator.

# I've manually created an edited PDB file which has all of the missing atoms. The 'dummy' atoms have coordinates
# (0,0,0) and b-factors of 0.
with open("6vfx_dummy.pdb", 'r') as f:
    lines = f.readlines()

# Let's look at an example dummy atom:
print(lines[995])

newlines = []

for line in lines:
    # Here we'll identify the lines corresponding to dummy atoms based on their b-factors of 0:
    if line[0:4] == "ATOM" and line[60:66] == "  0.00":
        # We need a random number for x, y, and z:
        num = tuple(uniform(-96.0, 96.0) for i in range(3))
        # And we'll replace the coordinates in the line with these properly-formatted numbers:
        line = line[:30] + "%8.3f" * 3 % num + line[54:]
        # Try to figure out how the above is equivalent to
        # line = line[:30] + "%8.3f" % num[0] + "%8.3f" % num[1] + "%8.3f" % num[2] + line[54:]
    newlines.append(line)

# Let's look at the same dummy atom as before:
print(newlines[995])

# Now we'll write out our edited PDB:
with open("6vfx_dummy2.pdb", 'w') as f:
    f.writelines(newlines)

ATOM   2005  N   LYS C 192       0.000   0.000   0.000  1.00  0.00           N

ATOM   2005  N   LYS C 192     -48.006  76.027  58.835  1.00  0.00           N



In [2]:
# The above method required a lot of manual effort. Let's see if we can automate adding the dummy residues using
# Bio.PDB:
from Bio import PDB as pdb

# First we'll retrieve the full mmCIF file from the PDB as before:
pdbl = pdb.PDBList()
filename = pdbl.retrieve_pdb_file('6vfx')
mmcifp = pdb.MMCIFParser()
structure_6vfx = mmcifp.get_structure('6vfx', filename)

# We'll get a bunch of errors about the 6 ClpX chains being discontinuous...

# We'll verify that we only have a single model in this structure. This example does not extend to multiple models.
print(list(structure_6vfx.get_models()))

Structure exists: '/home/kent/kuriyanlab_python-workshops/2/vf/6vfx.cif' 


[<Model id=0>]


/opt/anaconda/envs/demo/lib/python3.6/site-packages/Bio/PDB/StructureBuilder.py:92: PDBConstructionWarning: WARNING: Chain C is discontinuous at line 73193.
  PDBConstructionWarning,
/opt/anaconda/envs/demo/lib/python3.6/site-packages/Bio/PDB/StructureBuilder.py:92: PDBConstructionWarning: WARNING: Chain B is discontinuous at line 73237.
  PDBConstructionWarning,
/opt/anaconda/envs/demo/lib/python3.6/site-packages/Bio/PDB/StructureBuilder.py:92: PDBConstructionWarning: WARNING: Chain E is discontinuous at line 73281.
  PDBConstructionWarning,
/opt/anaconda/envs/demo/lib/python3.6/site-packages/Bio/PDB/StructureBuilder.py:92: PDBConstructionWarning: WARNING: Chain F is discontinuous at line 73325.
  PDBConstructionWarning,
/opt/anaconda/envs/demo/lib/python3.6/site-packages/Bio/PDB/StructureBuilder.py:92: PDBConstructionWarning: WARNING: Chain D is discontinuous at line 73364.
  PDBConstructionWarning,
/opt/anaconda/envs/demo/lib/python3.6/site-packages/Bio/PDB/StructureBuilder.py:92: P

In [3]:
# The Bio.PDB module itself does not have any tools for retrieving the SEQRES sequence from the header of
# PDB/mmCIF files (which should contain the sequence of regions that were not modeled as well). Instead, we'll use
# Bio.SeqIO.PdbIO, which is confusingly similar to Bio.PDB.PDBIO...

from Bio.SeqIO import PdbIO as pdbio

# We'll open the same mmCIF file as before and create a list of SeqRecords corresponding to individual chains:
with open(filename, 'r') as f:
    chainseqs = list(pdbio.CifSeqresIterator(f))
    
# Let's take a look at the list of sequences, and their lengths
print(chainseqs)
print([len(chain.seq) for chain in chainseqs])

[SeqRecord(seq=Seq('MSNENRTCSFCGKSKSHVKHLIEGENAFICDECVSNCIEILHEDGNDGTPSESA...FES', ProteinAlphabet()), id='6VFX:A', name='6VFX:A', description='UNP:A0A0Y4ZJG4 A0A0Y4ZJG4_NEIME', dbxrefs=['UNP:A0A0Y4ZJG4', 'UNP:A0A0Y4ZJG4_NEIME']), SeqRecord(seq=Seq('MSNENRTCSFCGKSKSHVKHLIEGENAFICDECVSNCIEILHEDGNDGTPSESA...FES', ProteinAlphabet()), id='6VFX:B', name='6VFX:B', description='UNP:A0A0Y4ZJG4 A0A0Y4ZJG4_NEIME', dbxrefs=['UNP:A0A0Y4ZJG4', 'UNP:A0A0Y4ZJG4_NEIME']), SeqRecord(seq=Seq('MSNENRTCSFCGKSKSHVKHLIEGENAFICDECVSNCIEILHEDGNDGTPSESA...FES', ProteinAlphabet()), id='6VFX:C', name='6VFX:C', description='UNP:A0A0Y4ZJG4 A0A0Y4ZJG4_NEIME', dbxrefs=['UNP:A0A0Y4ZJG4', 'UNP:A0A0Y4ZJG4_NEIME']), SeqRecord(seq=Seq('MSNENRTCSFCGKSKSHVKHLIEGENAFICDECVSNCIEILHEDGNDGTPSESA...FES', ProteinAlphabet()), id='6VFX:D', name='6VFX:D', description='UNP:A0A0Y4ZJG4 A0A0Y4ZJG4_NEIME', dbxrefs=['UNP:A0A0Y4ZJG4', 'UNP:A0A0Y4ZJG4_NEIME']), SeqRecord(seq=Seq('MSNENRTCSFCGKSKSHVKHLIEGENAFICDECVSNCIEILHEDGNDGTPSESA...FES

In [4]:
# We will need to convert from 1- to 3-letter amino acid codes, so we'll import a function from biopython to do so:
from Bio.SeqUtils import seq3

# Create a new structure, and add a new model with id 0 to it
structure_6vfx_edited = pdb.Structure.Structure('6vfx')
structure_6vfx_edited.add(pdb.Model.Model(0))

# We'll make a list of resolved chain lengths
chainlengths = [len(list(chain.get_residues())) for chain in structure_6vfx.get_chains()]
print(chainlengths[:7])

# And an empty list to keep track of the loops we will ask Rosetta to build
loops = []

# Now we want to iterate through the chains corresponding to ClpX (C,B,E,F,D,A), copying them to a new structure
for chainnum, chain in enumerate(structure_6vfx.get_chains()):
    chainid = chain.get_id()
    if chainid in "CBEFDA":
        # Residue.id is a tuple. From the source code:
        # (field - hetero flag; "W" for waters; "H" for hetero residues; otherwise blank,
        #  resseq - int; sequence identifier, icode - string; insertion code)
        resolvedlist = [residue.get_id()[1] for residue in chain.get_residues()]
        structure_6vfx_edited[0].add(pdb.Chain.Chain(chainid))
        for resnum, residue in enumerate(chain.get_residues()):
            structure_6vfx_edited[0][chainid].add(residue)
            if resnum < chainlengths[chainnum] - 3: # if not the last resolved residue
                if (residue.get_id()[1] + 1) not in resolvedlist: # if the next residue is not resolved
                    # count the number of residues already in the chain, as Rosetta numbers from 1
                    loopstart = len(list(structure_6vfx_edited.get_residues()))
                    buildid = residue.get_id()[1]
                    built = 0
                    print("building")
                    while buildid + 1 not in resolvedlist:
                        # Figure out next residue to build from sequence:
                        buildaa3 = seq3(chainseqs[chainnum].seq[buildid]).upper()
                        # find first example of that residue in the full structure and copy it
                        for candidateresnum, candidateresidue in enumerate(structure_6vfx.get_residues()):
                            if candidateresidue.get_resname() == buildaa3:
                                # make sure not to copy termini
                                if candidateresnum != 0 and candidateresnum < chainlengths[chainnum] - 3:
                                    # Now we will manually copy this residue and its atoms:
                                    tempresidue = pdb.Residue.Residue((' ', buildid + 1, ' '), buildaa3, ' ')
                                    for atom in candidateresidue.get_atoms():
                                        # randomize the coordinates, set b-factor to 0:
                                        tempatom = pdb.Atom.Atom(atom.get_name(), tuple(uniform(-96.0, 96.0) for i in range(3)), 0, 1, ' ', atom.get_fullname(), 0, atom.element)
                                        tempresidue.add(tempatom)
                                    continue # break out of for loop
                        # add residue to chain
                        structure_6vfx_edited[0][chainid].add(tempresidue)
                        buildid += 1
                        built += 1
                    # Add the start and end numbers based on Rosetta numbering
                    loops.append((loopstart, loopstart + built + 1))
    elif chainid == "G": # We'll also model the substrate, chain G, as a poly-alanine
        structure_6vfx_edited[0].add(pdb.Chain.Chain("G"))
        for residue in chain.get_residues():
            tempresidue = pdb.Residue.Residue(residue.get_id(), "ALA", ' ')
            for atom in residue.get_atoms():
                tempatom = pdb.Atom.Atom(atom.get_name(), atom.get_coord(), atom.get_bfactor(), 1, ' ', atom.get_fullname(), 0, atom.element)
                tempresidue.add(tempatom)
            structure_6vfx_edited[0]["G"].add(tempresidue)

print(loops)

[335, 339, 343, 322, 341, 342, 7]
building
building
building
building
building
building
building
building
building
building
building
building
building
building
[(129, 137), (165, 172), (212, 217), (515, 524), (564, 570), (834, 841), (918, 922), (1184, 1199), (1220, 1228), (1268, 1277), (1537, 1544), (1573, 1579), (1924, 1931), (1971, 1976)]


In [5]:
# Now we need to write our loops to the . Looking at the documentation, we want:
# LOOP start end (halfway between start and end) (0: never skip this loop) (1: discard initial loop conformation)
with open("loops.txt", 'w') as f:
    for loop in loops:
        f.write("LOOP " + str(loop[0]) + " " + str(loop[1]) + " " + str((loop[0] + loop[1]) // 2) + " 0 1\n")

In [6]:
# And write our edited structure to a PDB file:
pdbio = pdb.PDBIO()
pdbio.set_structure(structure_6vfx_edited)
pdbio.save("6vfx_edited.pdb")

In [7]:
# I have defined my own parameters for ATP more carefully than the default parameters, and I've called them A3P
# to avoid collision
# We'll quickly create a PDB file where we replace ATP with A3P:
with open("6vfx_edited.pdb", 'r') as f:
    lines = f.readlines()

with open("6vfx_edited2.pdb", 'w') as f:
    for line in lines:
        f.write(line.replace("ATP", "A3P"))

In [8]:
# Let's also create batch scripts using GNU parallel to run 16 processes with different random seeds on each node
# Here is an example batch script corresponding to the second job (processes 16-31)

"""
#!/bin/bash
#PBS -N rosetta_6vfx_1
#PBS -q shortq
#PBS -l nodes=1:ppn=16
#PBS -m ae
#PBS -M kent.gorday@berkeley.edu
#PBS -o rosetta_6vfx_1.o
#PBS -S /bin/bash

source /home/kentgorday/.bashrc

cd /home/kentgorday/rosetta_6vfx

parallel --tmpdir /export/silo/kent/scratch 'mkdir -p out/{} ; cd out/{} && /home/kentgorday/code/rosetta/main/source/bin/loopmodel.linuxgccrelease -database /home/kentgorday/code/rosetta/main/database -in:file:s ../../6vfx_edited2.pdb -in:file:fullatom -in:file:extra_res_fa /home/kentgorday/rosetta_ligands/A3P.fa.params /home/kentgorday/rosetta_ligands/ADP.fa.params -in:file:extra_res_cen /home/kentgorday/rosetta_ligands/A3P.cen.params /home/kentgorday/rosetta_ligands/ADP.cen.params -loops:loop_file ../../loop.txt -loops:remodel perturb_kic -loops:refine refine_kic -loops:taboo_sampling -loops:kic_rama2b -loops:ramp_fa_rep -loops:ramp_rama -ex1 -ex2 -overwrite -allow_omega_move true -loops:refine_outer_cycles 6 -nstruct 128 -run:constant_seed -run:jran {}' ::: {16..31}
"""

"\n#!/bin/bash\n#PBS -N rosetta_6vfx_1\n#PBS -q shortq\n#PBS -l nodes=1:ppn=16\n#PBS -m ae\n#PBS -M kent.gorday@berkeley.edu\n#PBS -o rosetta_6vfx_1.o\n#PBS -S /bin/bash\n\nsource /home/kentgorday/.bashrc\n\ncd /home/kentgorday/rosetta_6vfx\n\nparallel --tmpdir /export/silo/kent/scratch 'mkdir -p out/{} ; cd out/{} && /home/kentgorday/code/rosetta/main/source/bin/loopmodel.linuxgccrelease -database /home/kentgorday/code/rosetta/main/database -in:file:s ../../6vfx_edited2.pdb -in:file:fullatom -in:file:extra_res_fa /home/kentgorday/rosetta_ligands/A3P.fa.params /home/kentgorday/rosetta_ligands/ADP.fa.params -in:file:extra_res_cen /home/kentgorday/rosetta_ligands/A3P.cen.params /home/kentgorday/rosetta_ligands/ADP.cen.params -loops:loop_file ../../loop.txt -loops:remodel perturb_kic -loops:refine refine_kic -loops:taboo_sampling -loops:kic_rama2b -loops:ramp_fa_rep -loops:ramp_rama -ex1 -ex2 -overwrite -allow_omega_move true -loops:refine_outer_cycles 6 -nstruct 128 -run:constant_seed -r

In [9]:
constant1 = """#!/bin/bash
#PBS -N rosetta_6vfx_"""
# insert job # here
# Here we'll just assume 16 processors per node, but you could of course make this a variable as well
constant2 = """
#PBS -q shortq
#PBS -l nodes=1:ppn=16
#PBS -m ae
#PBS -M kent.gorday@berkeley.edu
#PBS -o rosetta_6vfx_"""
# insert job # here
constant3 = """.o
#PBS -S /bin/bash

source /home/kentgorday/.bashrc

cd /home/kentgorday/rosetta_6vfx

parallel --tmpdir /export/silo/kent/scratch 'mkdir -p out/{} ; cd out/{} && /home/kentgorday/code/rosetta/main/source/bin/loopmodel.linuxgccrelease -database /home/kentgorday/code/rosetta/main/database -in:file:s ../../6vfx_edited2.pdb -in:file:fullatom -in:file:extra_res_fa /home/kentgorday/rosetta_ligands/A3P.fa.params /home/kentgorday/rosetta_ligands/ADP.fa.params -in:file:extra_res_cen /home/kentgorday/rosetta_ligands/A3P.cen.params /home/kentgorday/rosetta_ligands/ADP.cen.params -loops:loop_file ../../loops.txt -loops:remodel perturb_kic -loops:refine refine_kic -loops:taboo_sampling -loops:kic_rama2b -loops:ramp_fa_rep -loops:ramp_rama -ex1 -ex2 -overwrite -allow_omega_move true -loops:refine_outer_cycles 6 -nstruct 128 -run:constant_seed -run:jran {}' ::: {"""
# insert start processes # for this job
constant4 = ".."
# insert end process # for this job
constant5 = "}"

numnodes = 2

for i in range(numnodes):
    with open("rosetta_6vfx_" + str(i) + ".pbs", 'w') as f:
        # putting it all together:
        f.write(constant1 + str(i) + constant2 + str(i) + constant3 + str(i * 16) + constant4 +
                str((i + 1) * 16 - 1) + constant5)